In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import cv2

import random
import math
import networkx as nx

import boto3
from PIL import Image

import requests
import json

from tqdm import tqdm
getattr(tqdm, '_instances', {}).clear()  # ⬅ add this line

# %matplotlib inline

plt.style.use('dark_background')

pd.set_option('display.max_columns', 50)

In [2]:
import pickle # save images
import time # get time stamp of models trained

import matplotlib.image as mpimg # show images
from io import BytesIO # reading bytes

In [3]:
# orders_df = pd.read_csv('data/orders_df.csv', index_col=0)

# orders_df.head(2)

# # sns.set()
# # sns.palplot(sns.dark_palette("purple"))

## Full Data Number of Species

In [ ]:
# fig, ax = plt.subplots(figsize=(10,10))

# ax = sns.countplot(orders_df['species_group'], order=orders_df.groupby('species_group').count()['family'].sort_values(ascending=False).index);
# plt.xticks(rotation=90);
# ax.set_title('Number of Species in Family Group')
# ax.set_xlabel('Family Groups')
# ax.set_ylabel('Number of Species')
# ax.grid(False)
# plt.tight_layout()

In [ ]:
# final_orders.to_csv('data/final_orders.csv')

## Read in final csv

In [4]:
final_orders = pd.read_csv('data/final_orders.csv', index_col=0)

In [5]:
final_orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21129 entries, 0 to 21128
Data columns (total 7 columns):
file_path        21129 non-null object
img_name         21129 non-null object
class_id         21129 non-null int64
txt_labels       21129 non-null object
order            21129 non-null object
family           21129 non-null object
species_group    21129 non-null object
dtypes: int64(1), object(6)
memory usage: 1.3+ MB


In [ ]:
np.unique(final_orders['species_group'])

In [ ]:
np.unique(final_orders['family'])

In [ ]:
group_sort = final_orders.groupby('species_group').count()['family'].sort_values(ascending=False)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

ax = sns.countplot(final_orders['species_group'], order=group_sort.index);
plt.xticks(rotation=90, fontsize=15);
ax.set_title('Number of Species in Family Group', fontsize=30)
ax.set_xlabel('Family Groups', fontsize=20)
ax.set_ylabel('Number of Species', fontsize=20)
ax.grid(False)

plt.tight_layout()

plt.savefig('graphs/num_fam_group.png')

In [ ]:
group_top5 = final_orders.groupby('species_group').count()['family'].sort_values(ascending=False)[:5]

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

ax = sns.barplot(x=group_top5.index.values, y=group_top5.values);
plt.xticks(rotation=90, fontsize=20);
ax.set_title('Number of Species in Family Group', fontsize=30)
ax.set_xlabel('Family Groups', fontsize=20)
ax.set_ylabel('Number of Species', fontsize=20)
ax.grid(False)
plt.tight_layout()

plt.savefig('graphs/top5_fam_group.png')

In [ ]:
order_sort = final_orders.groupby('order').count()['species_group'].sort_values(ascending=False)

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))

ax = sns.countplot(final_orders['order'], order=order_sort.index);
plt.xticks(rotation=90, fontsize=20);
ax.set_title('Number of Species per Order', fontsize=30)
ax.set_xlabel('Name of Order Group', fontsize=20)
ax.set_ylabel('Number of Species', fontsize=20)
ax.grid(False)
plt.tight_layout()

plt.savefig('graphs/num_species_order.png')

## Plot Num Species in Orders

In [ ]:
final_orders[final_orders['species_group']=='Cardinals, Grosbeaks, and Allies']['file_path'][1055:1057]

In [6]:
bucket = 'cwbirdsimages'
img_dir = 'images'

In [7]:
s3 = boto3.client('s3')

# grab duck image
obj = s3.get_object(Bucket=bucket, Key=f'images/0337/47b4bab08e524f0f9dff439c5386b70b.jpg')
img_bytes = BytesIO(obj['Body'].read())
waterfowl1 = Image.open(img_bytes)

# grab hawk image
obj = s3.get_object(Bucket=bucket, Key=f'images/0860/9593362c9fe54d1297356000a11f6aa5.jpg')
img_bytes = BytesIO(obj['Body'].read())
woodwarbler1 = Image.open(img_bytes)

# grab finch image
obj = s3.get_object(Bucket=bucket, Key=f'images/0988/914616d508cc46dd8b155c997c41b37f.jpg')
img_bytes = BytesIO(obj['Body'].read())
blackbird1 = Image.open(img_bytes)

# grab finch image
obj = s3.get_object(Bucket=bucket, Key=f'images/0899/77cf5ac8c6b243509384657586229871.jpg')
img_bytes = BytesIO(obj['Body'].read())
newworldsparrow1 = Image.open(img_bytes)

# grab finch image
obj = s3.get_object(Bucket=bucket, Key=f'images/0771/ed7362f8ad1a4939a2c89bc7ed81a283.jpg')
img_bytes = BytesIO(obj['Body'].read())
cardinals1 = Image.open(img_bytes)

In [ ]:
# fig, axes = plt.subplots(3, 4, figsize=(20,10));

# fig.suptitle('RGB Channels of Images', y=1.1, fontsize=20)

# axes[0][0].imshow(waterfowl1);
# axes[0][0].set_title('original')
# for ax, channel, name in zip(axes[0][1:], waterfowl1.split(), ['red channel', 'green channel', 'blue channel']):
#     ax.imshow(channel, cmap=f'{name.split()[0].capitalize()}s_r');
#     ax.set_title(name)
    
# axes[1][0].imshow(woodwarbler1);
# axes[1][0].set_title('original')
# for ax, channel, name in zip(axes[1][1:], woodwarbler1.split(), ['red channel', 'green channel', 'blue channel']):
#     ax.imshow(channel, cmap=f'{name.split()[0].capitalize()}s_r');
#     ax.set_title(name)
    
# axes[2][0].imshow(blackbird1);
# axes[2][0].set_title('original')
# for ax, channel, name in zip(axes[2][1:], blackbird1.split(), ['red channel', 'green channel', 'blue channel']):
#     ax.imshow(channel, cmap=f'{name.split()[0].capitalize()}s_r');
#     ax.set_title(name)

# plt.tight_layout()

In [ ]:
plt.title('New World Sparrow',fontsize=35)
plt.imshow(newworldsparrow1);
plt.savefig('graphs/newworldsparrow.png')

In [ ]:
order_fam = final_orders[final_orders['order']=='Passeriformes'].groupby('species_group').count()['order'].sort_values(ascending=False)

In [ ]:
order_fam.index.values

In [ ]:
plt.style.use('dark_background')

fig, ax = plt.subplots(figsize=(10,10))

ax = sns.barplot(x=order_fam.index.values, y=order_fam.values);
plt.xticks(rotation=90, fontsize=15);
ax.set_title('Number of Species in Passeriformes Order', fontsize=20)
ax.set_xlabel('Species Family Groups in Passeriformes', fontsize=15)
ax.set_ylabel('Number of Species', fontsize=15)
ax.grid(False)
plt.tight_layout()

plt.savefig('graphs/num_passer_order.png')

In [ ]:
img_dir = 'images' # folder containing all other folders of images
paths = orders_df['file_path']
bucket = 'cwbirdsimages'

def resize_images_array(img_dir, file_paths):
    # arrays of image pixels
    img_arrays = []
    
    paths = []
    
    # loop through the dataframe that is linked to its label so that all images are in the same order
    for path in tqdm(file_paths):
        s3 = boto3.client('s3')
        try:
            obj = s3.get_object(Bucket=bucket, Key=f'{img_dir}/{path}')
            img_bytes = BytesIO(obj['Body'].read())
            open_img = Image.open(img_bytes)
            arr = np.array(open_img.resize((299,299))) # resize to 200,200. possible to play around with better or worse resolution
            img_arrays.append(arr)
            paths.append(path)
        except:
#             print(path) # get file_path of ones that fail to load
            continue
    return np.array(img_arrays)

In [ ]:
# X = resize_images_array(img_dir, orders_df['file_path'][:21129])

In [ ]:
print('length of sample: ', len(X))
X.shape

In [ ]:
# normalize the RBG values
X = X/255.0

In [ ]:
# grab numeric label
# VALUES MUST BE NP.ARRAYS
label = np.array(orders_df['order'][:21129].values)

# labels are alphabetical
y = (label.reshape(-1,1) == np.unique(orders_df['order'][:21129])).astype(float)

In [ ]:
y[0]

In [ ]:
print('label shape: ', y.shape)
print('features shape: ', X.shape)

## Transfer Learning

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization # CNN
from tensorflow.keras.models import Model

from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications import Xception
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, RMSprop

from tensorflow.keras.callbacks import TensorBoard # graphical visual of loss and accuracy over the epochs of train and test set
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import datetime

tf.__version__

'2.2.0'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
print('X_train shape: ', X_train.shape)
print('X_test shape: ', X_test.shape)
print('y_train shape: ', y_train.shape)
print('y_test shape: ', y_test.shape)

In [ ]:
log_xcept = os.path.join("logs/orders_xception", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_xcept, histogram_freq=1)

In [ ]:
input_size = (299,299,3)

n_categories = y.shape[1]

In [28]:
def create_transfer_model(input_size, n_categories, weights = 'imagenet'):
        # note that the "top" is not included in the weights below
        base_model = Xception(weights=weights,
                          include_top=False,
                          input_shape=input_size)
        
        model = base_model.output
        model = GlobalAveragePooling2D()(model)
        predictions = Dense(n_categories, activation='softmax')(model)
        model = Model(inputs=base_model.input, outputs=predictions)
        
        return model

def change_trainable_layers(model, trainable_index):
    for layer in model.layers[:trainable_index]:
        layer.trainable = False
    for layer in model.layers[trainable_index:]:
        layer.trainable = True

In [ ]:
transfer_model = create_transfer_model((299,299,3),n_categories)

In [ ]:
_ = change_trainable_layers(transfer_model, 132)

In [ ]:
# print_model_properties(transfer_model, 130)

In [ ]:
transfer_model.compile(optimizer=RMSprop(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
xception_final = transfer_model.fit(X, y, batch_size=1000, epochs=5, validation_split=0.1, callbacks=[tensorboard_callback])

In [16]:
# %reload_ext tensorboard
# %tensorboard --logdir logs/

In [9]:
# transfer_model.save('saved_models/orders_xception.h5')
species_xception = tf.keras.models.load_model('saved_models/species3_xception.h5')

In [10]:
species_xception.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [ ]:
# pickle.dump(species_xception, open('saved_models/species_xception.p', 'wb'))

In [11]:
file = 'static/images/meet_duck.png'
a = Image.open(file)
c = np.array(a.resize((299,299)))

In [12]:
ans = species_xception.predict(c.reshape(1,299,299,3))

In [13]:
ans

array([[2.0544770e-36, 1.8704508e-29, 6.2675882e-31, 0.0000000e+00,
        0.0000000e+00, 1.8498242e-28, 4.7007343e-29, 7.8685414e-25,
        0.0000000e+00, 7.1845735e-36, 3.3938067e-27, 3.5663827e-25,
        1.1993901e-36, 1.0199423e-28, 0.0000000e+00, 0.0000000e+00,
        1.4651046e-32, 0.0000000e+00, 2.0795709e-32, 5.9080591e-31,
        0.0000000e+00, 3.0493652e-22, 3.5587822e-35, 4.1716126e-38,
        1.3497117e-33, 1.0000000e+00, 5.0679876e-38, 3.4350064e-33,
        0.0000000e+00, 0.0000000e+00, 4.1084264e-23, 0.0000000e+00,
        0.0000000e+00, 8.1804222e-34, 0.0000000e+00, 6.8899181e-27,
        5.1736068e-30, 6.3068881e-34, 0.0000000e+00]], dtype=float32)

In [14]:
np.round(ans * 100, 1)[0].argsort()[-1:-4:-1]

array([25, 38, 18])

In [17]:
conf_mat = pd.read_csv('data/confusion_matrix.csv', header=None)

In [33]:
confusion_mat = []

for i in conf_mat[0]:
    row = np.array(i.split(' ')).astype(float)
    confusion_mat.append(row)

In [34]:
confusion_mat

[array([31.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 11.,  0.,  0.,  0.]),
 array([  0., 219.,  21.,   3.,   0.,   4.,   0.,   7.,   0.,   0.,   0.,
          0.,   2.,   2.,   0.,   0.,   0.,   0.,   0.,   8.,   0.,   2.,
          0.,   0.,   0.,   0.,   0.,   3.,   0.,   1.,   1.,  18.,   0.,
          5.,   1.,   2.,  22.,   0.,   1.]),
 array([  0.,   6., 188.,   0.,   0.,   0.,   0.,   3.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   2.,  11.,   0.,   0.,
          0.,   0.,   1.,   0.,   0.,   0.,   0.,   3.,   0.,  13.,   1.,
          0.,   1.,   0.,  17.,   0.,   2.]),
 array([ 0.,  6.,  0., 17.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         1.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  0.,  3.,  3.,  3.,  1.,  0.,  0.,  0.,  1.,  1.,  0.]),
 array([ 0.,  2.,  0.,

In [ ]:
plot_confusion_matrix(conf_mat=confusion_mat, figsize=(8,8), class_names=folders);

In [ ]:
acc_loss = pd.read_csv('data/accuracy.csv', index_col=0)

In [ ]:
acc_loss

In [ ]:
# fig, axes = plt.subplots(1,2,figsize=(12,6))

# epoch_range = range(1, 11)

# axes[0].plot(epoch_range, acc_loss['accuracy'])
# axes[0].plot(epoch_range, acc_loss['val_accuracy'])
# axes[0].set_ylabel('Accuracy')
# axes[0].set_xlabel('Number of Epochs')
# axes[0].legend(['Train', 'Val'], loc='upper left')
# axes[0].set_title('Model2 Accuracy')

# axes[1].plot(epoch_range, acc_loss['loss'])
# axes[1].plot(epoch_range, acc_loss['val_loss'])
# axes[1].set_ylabel('Accuracy')
# axes[1].set_xlabel('Number of Epochs')
# axes[1].legend(['Train', 'Val'], loc='upper left')
# axes[1].set_title('Model2 Loss')

# plt.savefig('graphs/model2_acc_loss.png')

In [ ]:
acc = acc_loss['accuracy']
val_acc = acc_loss['val_accuracy']

loss = acc_loss['loss']
val_loss = acc_loss['val_loss']

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.plot(acc, label='Training Accuracy')
ax.plot(val_acc, label='Validation Accuracy')
ax.legend(loc='lower right')
ax.set_ylabel('Accuracy', fontsize=15)
ax.set_xlabel('Number of Epoch', fontsize=15)
ax.set_ylim([min(plt.ylim()),1])
ax.grid(False)
plt.title('Training and Validation Accuracy', fontsize=20)
plt.savefig('graphs/xception_acc.png')

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.plot(loss, label='Training Loss')
ax.plot(val_loss, label='Validation Loss')
ax.legend(loc='lower left')
ax.set_ylabel('Cross Entropy', fontsize=15)
ax.set_xlabel('Number of Epoch', fontsize=15)
ax.set_ylim([min(plt.ylim()),1])
ax.grid(False)
plt.title('Training and Validation Loss', fontsize=20)
plt.savefig('graphs/xception_loss.png')

In [47]:
makeup=np.array([[0.3,0.5,0.1,0.7,0.3,0.8],[0.9,0.5,0.1,0.7,0.3,0.8]])

In [55]:
for i in makeup:
    i[i.argmax()] = 1
    i[i < 1] = 0
    print(i)

[0. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 0.]


In [48]:
makeup[makeup.argmax(axis=1)] = 1

IndexError: index 5 is out of bounds for axis 0 with size 2

In [41]:
makeup[makeup < 1] = 0

In [42]:
makeup

array([0., 0., 0., 0., 0., 1.])

In [68]:
np.loadtxt('data/sk_report.npy', encoding='ISO-8859–1', error_bad_lines=False)

TypeError: loadtxt() got an unexpected keyword argument 'error_bad_lines'

In [64]:
testing = pd.read_csv('data/sk_report.npy', sep='\n', encoding='ISO-8859–1')

In [65]:
testing

,NUMPY
0,
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,NaN
